In [1]:
with open("./input.txt", "r") as file: 
    data = [
        chunk.split("\n") for chunk 
        in file.read().strip().split("\n\n")
    ]

# Part 1

In [2]:
import re
import tqdm
import functools

class Monkey: 
    def __init__(self, name, items, operate, divisor, outcomes): 
        """
        Parameters
        
        items : list[int]
            List of items
        operate : func
            Transforms the item: lambda item: item
        test : func
            Tests the item : lambda item: True
        outcomes : dict
            Mapping of outcomes: {True: xx, False: yy}
        """
        self.name     = name
        self.items    = items
        self.operate  = operate
        self.divisor  = divisor
        self.outcomes = outcomes
        self.count    = 0 
    
    def __repr__(self):
        return f"{self.name}: {','.join([str(i) for i in self.items])} ({self.count} inspections)"
    
    def append(self, value):
        self.items.append(value)
    
    def test(self, value):
        return value % self.divisor == 0
    
    def play(self, peers):
        while len(self.items) > 0: 
            item = self.operate(self.items.pop(0))
            peers[self.outcomes[self.test(item)]].append(item)
            self.count += 1
            
        return self
    
    @classmethod
    def parse(cls, data, level=1): 
        name  = data[0][:-1]
        items = [int(x) for x in data[1].split(":")[1].split(",")]
        
        match = re.search(
            "Operation: new = old (?P<operation>[\*\+]) (?P<operand>(\d+|old))", 
            data[2]
        )
        
        operations = {
            "+":lambda x, y: x + y,
            "*":lambda x, y: x * y
        }
        
        postoperate = {
            1:lambda x: int(x / 3),
            2:lambda x: x
        }
        
        operate = lambda value: postoperate[level](
            operations[match.group("operation")](
                value, 
                value if match.group("operand") == "old" else int(match.group("operand"))
            )
        )
        
        divisor = int(data[3].split(" ")[-1])
        
        outcomes = {
            True: f"Monkey {data[4][-1]}",
            False: f"Monkey {data[5][-1]}"
        }
        
        return cls(name, items, operate, divisor, outcomes)
        
            
class Game: 
    def __init__(self, monkeys):
        self.monkeys = {monkey.name:monkey for monkey in monkeys}
        
    def play(self, rounds=1): 
        for r in tqdm.tqdm(range(rounds)):
            for monkey in self.monkeys.values():
                monkey.play(self.monkeys)
                
        return self
    
    @classmethod
    def parse(cls, data):
        monkeys = []
        for chunk in data: 
            monkeys.append(Monkey.parse(chunk, level=1))
        return cls(monkeys)
    
    @property
    def score(self):
        return functools.reduce(
            lambda acc, curr : acc * curr.count, 
            sorted(self.monkeys.values(), key=lambda m: m.count, reverse=True)[0:2],
            1
        )
        
game = Game.parse(data)
game.play(rounds=20)
game.score

100%|████████████████████████████████████████| 20/20 [00:00<00:00, 10297.82it/s]


88208

In [3]:
class Divisible: 
    @classmethod
    def create(cls, value, divisors):
        return cls(
            [value % divisor for divisor in divisors], 
            divisors
        )
    
    def __init__(self, remainders, divisors): 
        self.remainders = remainders
        self.divisors   = divisors
    
    def __add__(self, value):
        remainders = [
            (value + remainder) % divisor 
            for remainder, divisor 
            in zip(self.remainders, self.divisors)
        ]
        
        return Divisible(remainders, self.divisors)
        
    def __mul__(self, value): 
        remainders = [
            (value * remainder) % divisor
            for remainder, divisor 
            in zip(self.remainders, self.divisors)
        ]
        
        return Divisible(remainders, self.divisors)
    
    def __mod__(self, divisor): 
        if divisor not in self.divisors: 
            raise ValueError(f"Divisor {divisor} not in number divisors")
        return self.remainders[self.divisors.index(divisor)]
    
    def __repr__(self):
        representation = ", ".join(f"{divisor}:{remainder}" for remainder, divisor 
            in zip(self.remainders, self.divisors))
        
        return f"<Divisible '{representation}'>"
    
class Newgame(Game): 
    def __init__(self, monkeys): 
        divisors = [monkey.divisor for monkey in monkeys]
        
        for monkey in monkeys: 
            monkey.items = [Divisible.create(i, divisors) for i in monkey.items]
        
        super().__init__(monkeys)
        
    @classmethod
    def parse(cls, data):
        monkeys = []
        for chunk in data: 
            monkeys.append(Monkey.parse(chunk, level=2))
        return cls(monkeys)
    
    def __repr__(self):
        return "\n".join([str(monkey) for monkey in self.monkeys.values()])


In [4]:
game = Newgame.parse(data)
game.play(10000)
game.score

100%|███████████████████████████████████| 10000/10000 [00:03<00:00, 3103.99it/s]


21115867968